In [1]:
import torch
import numpy as np
import sys
import spatial_utils.data_process as sudp
import spatial_utils.data_read as sudr
import spatial_utils.feature_extract as sufe

# 好像不清楚滑窗截取的数据存在哪儿了，重新整一下
import tqdm
from datetime import datetime
import os
import pandas as pd
import time
import tracemalloc

eeg_trials = []
labels = []
de_features = []
subject_indexs = [0]
eeg_ex_index = []
subject_lists = [
    i for i in range(1, 36) if i != 11 and i != 9 and i != 19 and i != 20 and i != 33
]

In [2]:
eye_data = sudr.get_eye_track_trials_4(12)
eye_labels = sudr.get_labels(12)

/mnt/nvme1/yihaoyuan/SpatialAbilityTest/eeg_generated_from_eye/torch_base/data/spatial_utils/data_read.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial["Pupil diameter left"] = trial["Pupil diameter left"].fillna(
/mnt/nvme1/yihaoyuan/SpatialAbilityTest/eeg_generated_from_eye/torch_base/data/spatial_utils/data_read.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial["Pupil diameter right"] = trial["Pupil diameter right"].fillna(
/mnt/nvme1/yihaoyuan/SpatialAbilityTest/eeg_generated_from_e

In [9]:
data = np.concatenate(eye_data, axis=0)
print(data.shape)
new_data, new_label = sudp.re_data_slide(
    eye_data[0], eye_labels[0], win_len=60, overlap=0, is_filter=None, norm_method=None
)
print(new_data.shape)
print(new_label.shape)

(59340, 4)
(14, 60, 4)
(14, 3)


In [4]:
eeg_ex_index = np.load("/data/SpaticalTest/Spatial/total_33_eeg_ex_index.npy")
print(eeg_ex_index[9])
eeg_ex_index.shape

[  0  16  26  37  55  75  86 103 122 137 148 163 175 188 204 217 234 246
 263 280 296 313 321 337 349 369 381 396 412 422 442 469 494 508 525 555
 576 592 608 629 646 664 680 700 714 731 754 779 798 817 832 846 862 881
 889]


(30, 55)

In [7]:
import torch.nn.functional as F
per_eye_data = []
per_eye_label = []
for i, (data, label) in enumerate(zip(eye_data, eye_labels)):
    if i == 7:
        new_data, new_label = sudp.re_data_slide(
            data, label, win_len=60, overlap=0, is_filter=None, norm_method=None
        )
        # 均值补齐
        eeg_len = eeg_ex_index[9][i + 1] - eeg_ex_index[9][i]
        diff = eeg_len - len(new_data)
        if diff > 0:
            new_data = np.pad(new_data,((0,diff),(0,0),(0,0)),mode='mean')
        elif diff < 0:
            new_data = new_data[:diff]
            eeg_len = len(new_data)

        new_label = np.array([label] * eeg_len )
        print(i, len(new_data), len(new_label))
        per_eye_data.append(new_data)
        per_eye_label.append(new_label)
per_eye_data = np.concatenate(per_eye_data)
per_eye_label = np.concatenate(per_eye_label)
print(per_eye_data.shape)

KeyboardInterrupt: 

In [41]:
total_eye_data = []
total_eye_label = []
for ids, per_no in enumerate(subject_lists):
    print(f"Processing {per_no} ...")
    per_eye_data = []
    per_eye_label = []
    eye_data = sudr.get_eye_track_trials_4(per_no)
    eye_labels = sudr.get_labels(per_no)
    for i, (data, label) in enumerate(zip(eye_data, eye_labels)):
        new_data, new_label = sudp.re_data_slide(
            data, label, win_len=60, overlap=0, is_filter=None, norm_method=None
        )
        # 均值补齐
        eeg_len = eeg_ex_index[ids][i + 1] - eeg_ex_index[ids][i]
        diff = eeg_len - len(new_data)
        if diff > 0:
            new_data = np.pad(new_data,((0,diff),(0,0),(0,0)),mode='mean')
        elif diff < 0:
            new_data = new_data[:diff]
            eeg_len = len(new_data)

        new_label = np.array([label] * eeg_len )
        # print(i, len(new_data), len(new_label))
        per_eye_data.append(new_data)
        per_eye_label.append(new_label)
    per_eye_data = np.concatenate(per_eye_data)
    per_eye_label = np.concatenate(per_eye_label)
    print(f"{per_no} per_eye_data.shape: {per_eye_data.shape}")
    print(f"{per_no} per_eye_label.shape: {per_eye_label.shape}")
    total_eye_data.append(per_eye_data)
    total_eye_label.append(per_eye_label)
total_eye_data = np.concatenate(total_eye_data)
total_eye_label = np.concatenate(total_eye_label)
print(total_eye_data.shape)
print(total_eye_label.shape)

Processing 1 ...
1 per_eye_data.shape: (1051, 60, 4)
1 per_eye_label.shape: (1051, 3)
Processing 2 ...
2 per_eye_data.shape: (1696, 60, 4)
2 per_eye_label.shape: (1696, 3)
Processing 3 ...
3 per_eye_data.shape: (2648, 60, 4)
3 per_eye_label.shape: (2648, 3)
Processing 4 ...
4 per_eye_data.shape: (3416, 60, 4)
4 per_eye_label.shape: (3416, 3)
Processing 5 ...
5 per_eye_data.shape: (4314, 60, 4)
5 per_eye_label.shape: (4314, 3)
Processing 6 ...
6 per_eye_data.shape: (5161, 60, 4)
6 per_eye_label.shape: (5161, 3)
Processing 7 ...
7 per_eye_data.shape: (6222, 60, 4)
7 per_eye_label.shape: (6222, 3)
Processing 8 ...
8 per_eye_data.shape: (7099, 60, 4)
8 per_eye_label.shape: (7099, 3)
Processing 10 ...
10 per_eye_data.shape: (8064, 60, 4)
10 per_eye_label.shape: (8064, 3)
Processing 12 ...


ValueError: can't extend empty axis 0 using modes other than 'constant' or 'empty'